<img src="https://www.rp.edu.sg/images/default-source/default-album/rp-logo.png" width="200" alt="Republic Polytechnic"/>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/koayst-rplesson/SST_DP2025/blob/main/Day_02/L10/L10_Answer.ipynb)

# Setup and Installation

You can run this Jupyter notebook either on your local machine or run it at Google Colab.

* For local machine, it is recommended to install Anaconda and create a new development environment called `SST_DP2025`.
* Pip/Conda install the libraries stated below when necessary.
---

# <font color='red'>ATTENTION</font>

## Google Colab
- If you are running this code in Google Colab, **DO NOT** store the API Key in a text file and load the key later from Google Drive. This is insecure and will expose the key.
- **DO NOT** hard code the API Key directly in the Python code, even though it might seem convenient for quick development.
- You need to enter the API key at python code `getpass.getpass()` when ask.

## Local Environment/Laptop
- If you are running this code locally in your laptop, you can create a env.txt and store the API key there.
- Make sure env.txt is in the same directory of this Jupyter notebook.
- You need to install `python-dotenv` and run the Python code to load in the API key.

---
```
%pip install python-dotenv

from dotenv import load_dotenv

load_dotenv('env.tx')
openai_api_key = os.getenv('OPENAI_API_KEY')
```
---

## GitHub/GitLab
- **DO NOT** `commit` or `push` API Key to services like GitHub or GitLab.



# Lesson 10

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain
%pip install --quiet -U langgraph
%pip install --quiet -U langchain-openai
%pip install --quiet -U grandalf

In [ ]:
# grandalf         0.8
# langchain        0.3.11
# langgraph        0.2.59
# langchain-core   0.3.24
# langchain-openai 0.2.12
# openai           1.57.2
# pydantic         2.10.3

In [1]:
import getpass
import os

# setup the OpenAI API Key

# get OpenAI API key ready and enter it when ask
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


## Chains

### Basic/Entry Chain
A basic chain or entry chain using LCEL.

In [2]:
# load langchain libraries

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate

In [3]:
prompt = ChatPromptTemplate.from_template(
    "How do I say 'Hello' in {language}?",
) 

In [4]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0.5,
)

In [5]:
print(model.model_name)

gpt-4o-mini


In [6]:
chain = prompt | model | StrOutputParser()

In [7]:
response = chain.invoke({"language" : "French"})

print(response)

You say 'Hello' in French as "Bonjour."


### Simple Sequential Chain (Single Input / Single Output)
Simple chain where the output of one step feed directly into next. A streamlined version of a sequential chain, where the output of each step directly becomes the input for the next. It is perfect for straightforward workflows without the need for advanced memory or branching logic.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from typing import Dict

In [9]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0.5,
)

In [10]:
first_prompt = ChatPromptTemplate.from_template(
    "What is a good name for a company that specialises in making {product}?"
)

second_prompt = ChatPromptTemplate.from_template(
    "Generate a short introduction of company: {company} ."
)

In [11]:
chain = (
    first_prompt
    | model
    | {'company' : RunnablePassthrough()}   # 'company' is generated after the first prompt
    | second_prompt
    | model
    | StrOutputParser()
).with_types(input_type=Dict[str, str], output_type=str)

In [12]:
chain.invoke({'product' : 'Running shoe'})

"**Welcome to StridePro** – where every step counts! At StridePro, we specialize in high-performance running shoes designed to enhance your running experience. Our mission is to empower runners of all levels, from beginners to seasoned marathoners, with innovative footwear that combines comfort, durability, and cutting-edge technology. With a commitment to quality and style, our shoes are crafted to support your journey, whether you're hitting the pavement or conquering trails. Join us in celebrating the joy of running and discover your perfect stride with StridePro!"

In [13]:
# display the chain as a graph

chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *       

### Sequential Chain
The `Sequential Chain` in LangChain is a simple way to execute a series of tasks in order, where the output of one step becomes the input for the next. This type of chain is ideal for workflows where the steps depend on one another in a linear sequence.

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [15]:
first_prompt = ChatPromptTemplate.from_template(
    "Write a brief introduction about {topic}."
)

first_chain = first_prompt | model 

In [16]:
second_prompt = ChatPromptTemplate.from_template(
    "From this introduction: \"{introduction}\", generate three key bullet points."
)

second_chain = second_prompt | model

In [17]:
# observe in this prompt, it takes in two inputs 'introduction' and 'bullet_points' from earlier chains

third_prompt = ChatPromptTemplate.from_template(
    "Combine the introduction: \"{introduction}\" with the key points: \"{bullet_points}\".\n"
    "Write a concise summary from this information."
)

third_chain = third_prompt | model

In [18]:
chain = (
    {"introduction" : first_chain}
    | RunnablePassthrough.assign(bullet_points=second_chain)
    | RunnablePassthrough.assign(summary=third_chain)
)

In [19]:
response = chain.invoke({"topic" : "Artificial Intelligence"})

In [20]:
print(response['summary'].content)

**Summary of Artificial Intelligence (AI)**

Artificial Intelligence (AI) involves the simulation of human intelligence processes by machines, including learning, reasoning, problem-solving, perception, and language understanding. AI technologies, such as machine learning, neural networks, and natural language processing, are increasingly integrated into various sectors like healthcare, finance, transportation, and entertainment, significantly enhancing efficiency and innovation. However, the evolution of AI also raises important ethical and societal questions about its impact on jobs, privacy, and decision-making, highlighting the need for careful consideration of its implications.


In [21]:
chain.get_graph().print_ascii()

              +-----------------------------+           
              | Parallel<introduction>Input |           
              +-----------------------------+           
                             *                          
                             *                          
                             *                          
                  +--------------------+                
                  | ChatPromptTemplate |                
                  +--------------------+                
                             *                          
                             *                          
                             *                          
                      +------------+                    
                      | ChatOpenAI |                    
                      +------------+                    
                             *                          
                             *                          
                             * 

In [22]:
# Inspecting Runnables

chain.get_prompts()

[ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Write a brief introduction about {topic}.'), additional_kwargs={})]),
 ChatPromptTemplate(input_variables=['introduction'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['introduction'], input_types={}, partial_variables={}, template='From this introduction: "{introduction}", generate three key bullet points.'), additional_kwargs={})]),
 ChatPromptTemplate(input_variables=['bullet_points', 'introduction'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['bullet_points', 'introduction'], input_types={}, partial_variables={}, template='Combine the introduction: "{introduction}" with the key points: "{bullet_points}".\nWrite a concise summary

### Routing between Sub-Chains
This can be another form of router chain implementation.

In [23]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [24]:
prompt = PromptTemplate.from_template(
"""Given the user question below, classify it as either being about `Math`, `Science`, or `General`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
)

In [25]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
)

In [26]:
chain = prompt | model | StrOutputParser()

In [27]:
# test and ensure the classification is working

chain.invoke({"question" : "What is the basic theory of quantum physics?"})

'Science'

In [28]:
# create a math chain

math_chain = PromptTemplate.from_template(
"""You are an expert in Mathematics.
Always answer questions starting with "As my MATH teacher told me".
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model = "gpt-4o-mini")

In [29]:
# create a science chain

science_chain = PromptTemplate.from_template(
"""You are an expert in Science.
Always answer questions starting with "As my SCIENCE teacher told me".
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model = "gpt-4o-mini")

In [30]:
# create a generic chain

general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model = "gpt-4o-mini")

In [31]:
# define a custom function to route between different outputs

def route(info):
    if "math" in info['topic'].lower():
        return math_chain
    elif "science" in info['topic'].lower():
        return science_chain
    else:
        return general_chain

In [32]:
# create the full chain

full_chain = (
    {"topic" : chain, "question" : lambda x: x['question']} 
    | RunnableLambda (route)
)

In [33]:
response = full_chain.invoke({"question" : "What is the basic theory of quantum physics?"})

print(response.content)

As my SCIENCE teacher told me, the basic theory of quantum physics revolves around the idea that energy and matter exist in discrete units, or "quanta." This theory fundamentally challenges classical physics by introducing concepts such as wave-particle duality, where particles like electrons can exhibit properties of both particles and waves. Additionally, quantum physics emphasizes the role of probability and uncertainty, encapsulated in Heisenberg's Uncertainty Principle, which states that certain pairs of physical properties, like position and momentum, cannot be simultaneously known to arbitrary precision. Overall, quantum physics provides a framework for understanding the behavior of matter and energy at the smallest scales, leading to revolutionary advancements in technology and our understanding of the universe.


In [34]:
response = full_chain.invoke({"question" : "What is Merlion?"})

print(response.content)

The Merlion is a famous national symbol and landmark of Singapore. It is a mythical creature with the head of a lion and the body of a fish. The lion's head represents Singapore's original name, Singapura, which means "Lion City" in Malay, while the fish body symbolizes Singapore's origins as a fishing village. The Merlion statue stands at 8.6 meters (28 feet) tall and is often depicted spouting water from its mouth into Marina Bay. It has become an iconic representation of Singapore, attracting millions of tourists and serving as a popular photo opportunity. The Merlion has various replicas and is associated with the country's identity and tourism.


In [35]:
full_chain.get_prompts()

[PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Given the user question below, classify it as either being about `Math`, `Science`, or `General`.\n\nDo not respond with more than one word.\n\n<question>\n{question}\n</question>\n\nClassification:'),
 PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an expert in Mathematics.\nAlways answer questions starting with "As my MATH teacher told me".\nRespond to the following question:\n\nQuestion: {question}\nAnswer:'),
 PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an expert in Science.\nAlways answer questions starting with "As my SCIENCE teacher told me".\nRespond to the following question:\n\nQuestion: {question}\nAnswer:'),
 PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Respond to the following question:\n\nQuestion: {question}\nAnswer:')]

## Runnable Interface
This `Runnable` interface provides a standard way to create modular and resuable components in a chain or pipeline. It defines the behavior of components that can process inputs and produce outputs, ensuring interoperability across LangChain's ecosystem

In [36]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [37]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
)

In [38]:
first_prompt = ChatPromptTemplate.from_template("""
You are an expert AI tweet generator. 
Observe the user's prompt, and generate a witty tweet, and include emojis and hashtags.

Prompt: {prompt}
Tweet: """
)

- A simple `tweet_generator` is composed by chaining together a prompt, model and an output parser in sequence.
- Chaining is possible because the components (prompt, model and output parser) implemented the `runnable` interface.

In [39]:
tweet_generator = first_prompt | model | StrOutputParser()
tweet = tweet_generator.invoke({"prompt": "Langchain releases LangGraph for building stateful, multi-action applications. https://langchain.com"})

In [40]:
second_prompt = ChatPromptTemplate.from_template("""
You are an expert in AI tweet fixer.
Fix user's original tweet and based it on the mood.

Original Tweet: {tweet}
Mood: {mood}
Fixed Tweet:"""
)

In [41]:
tweet_fixer = second_prompt | model | StrOutputParser()
fixed_tweet = tweet_fixer.invoke({"tweet" : tweet, "mood" : "funny"})

In [42]:
fixed_tweet

"🚀 Hold onto your keyboards, developers! 🌟 Langchain just unleashed LangGraph, your new coding sidekick for building stateful, multi-action apps! It's like a Swiss Army knife, but for code! 🛠️💻 Time to level up and impress your cat with your skills! 😸✨ Check it out 👉 https://langchain.com #LangGraph #DevLife #CodingMagic"

### RunnableParallel
Execute more than one `Runnable` components concurrently. It is particularly useful for tasks that can be executed independently, as it improves efficiency by running these tasks in parallel.

In [43]:
chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model

In [44]:
chain2 = ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}") | model

In [45]:
from langchain_core.runnables.base import RunnableParallel

parallel_chain = RunnableParallel(joke=chain1, short_poem=chain2)

In [46]:
%time

response = parallel_chain.invoke({"topic" : "bears"})

# the time measured is too tiny to tell the difference between parallel or batch chain

CPU times: total: 0 ns
Wall time: 0 ns


In [47]:
response

{'joke': AIMessage(content='Why do bears have sticky paws?\n\nBecause they always paws for a moment before they swipe!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-a79454db-e0d9-4bf8-bfe7-4cbef0c86cc7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'short_poem': AIMessage(content="In forest shadows, mighty bears roam free,  \nNature's gentle giants, wild as they can be.", additional_kwargs={'refusal': None}, response_met

In [48]:
response['joke'].content

'Why do bears have sticky paws?\n\nBecause they always paws for a moment before they swipe!'

In [49]:
response['short_poem'].content

"In forest shadows, mighty bears roam free,  \nNature's gentle giants, wild as they can be."

In [50]:
parallel_chain.get_graph().print_ascii()

            +--------------------------------+             
            | Parallel<joke,short_poem>Input |             
            +--------------------------------+             
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+*                     +------------+     
                   ***               ***                   
                      ***         ***   

### RunnablePassthrough
Serves as a simple utility for passing data through without modification. It is useful in a pipeline or chain of operations when you want to retain certain intermediate results or bypass specific steps without applying transformations.

In [51]:
from operator import itemgetter

In [52]:
# "mood" and "prompt" are passed as input to "tweet_chain"
# RunnablePassthrough() makes both of these passes through unchanged
# itemgetter extracts the value of mood parameter and assigned it to "mood"

tweet_chain = RunnableParallel(
    {
        "mood": RunnablePassthrough() | itemgetter("mood"),
        "tweet": tweet_generator
    }
) | tweet_fixer

In [53]:
tweet_chain.invoke(
    {
        "prompt": "Langchain releases LangGraph for building stateful, multi-action applications. https://langchain.com",
        "mood": "sarcastic"
    }
)

'Oh great, just what we needed—another tool! 🙄 Langchain launched LangGraph, your “best friend” for building stateful, multi-action apps. Because clearly, our coding game wasn’t already complicated enough! 💻✨ Check it out if you dare: https://langchain.com #Langchain #LangGraph #CodingOverload #AppDevelopment'

### Batch
Using this method can significantly improve performance when needing to process multiple independent inputs as the processing can be done in parallel instead of sequentially.

In [54]:
%time

response = parallel_chain.batch([{"topic": "bears"}, {"topic": "cats"}])

# the time measured is too tiny to tell the difference between parallel or batch chain

CPU times: total: 0 ns
Wall time: 0 ns


In [55]:
response[0]['joke'].content

'Why do bears have hairy coats?\n\nBecause they look silly in sweaters!'

In [56]:
response[0]['short_poem'].content

'In forests deep where shadows play,  \nMighty bears roam wild, both fierce and gray.'

In [57]:
response[1]['joke'].content

'Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!'

In [58]:
response[1]['short_poem'].content

'Silent paws on moonlit floors,  \nWhiskered dreams behind closed doors.'

### Stream
Streaming enhances the responsiveness of application by displaying the output progressively, even before a complete response is ready. Due the latency of LLMs response generation, streaming improves user experience (UX).

In [59]:
joke_str = ""
short_poem_str = ""

for s in parallel_chain.stream({"topic" : "dog"}):
    if 'joke' in s:
        joke_str = joke_str + s['joke'].content + " | "
    elif 'short_poem' in s:
        short_poem_str = short_poem_str + s['short_poem'].content + " | "

print(joke_str)
print('-'*10)
print(short_poem_str)

 | Why |  did |  the |  dog |  sit |  in |  the |  shade | ?

 | Because |  he |  didn | ’t |  want |  to |  become |  a |  hot |  dog | ! |  | 
----------
 | In |  fields |  of |  green | , |  a |  joyful |  chase | , |   
 | A |  wag | ging |  tail | , |  love | 's |  pure |  embrace | . |  | 
